# 최종 정리

## 요구사항

이 것을 왜 하냐? 계속 구글 스프레스 시트로 반복 작업을 하고 싶지 않았기 때문이다.

- Ajax 로 특정 시군구 긁어오기
- 정렬, 복사, 붙여넣기 등
- 스프레트 시트에 지역-탭 별로 나누기

이 정보의 최종 활용자는? 50대 컴퓨터에 익숙치 않은 어른. 그가 이 프로그램을 조작하진 않는다. 그리고 난 그에게 엑셀 파일만 주면 된다.

몇 개의 필드를 숨기거나 하는 작업을 해도 된다. 그렇지만, 자료를 업데이트할 때마다 매번 자료의 형태를 가공해야하는 작업이 싫다.

즉, 최종적으로 나는

1. 내가 원하는 지역 경기도, 인천, 서울의 모든 아파트 정보를
    - 각 페이지에
    - 함수 한 번으로 가져오고
    - 저장한 다음에
    - 이것을 아래의 형태로 가공해서 각 부천,시흥시,광명시,구로구,양천구,부평구 페이지에 저장한다.
    - 양식을 붙여넣을 수 있기 때문에.

2. 최신 정보를 갱신할 때는.
    - 기존의 정보의 명칭과 새로 받아온 것의 명칭을 비교해서
    - 정보가 틀릴 때는 최신 것으로 업데이트하고
    - 정보가 없을 때는 새로 추가하며
    - 정보가 같을 때는 아무일도 하지 않는다.

그렇게 되면 나는 이미 가공된 자료의 필드 몇개를 숨기거나 양식을 붙여넣는 수준으로 작업을 편하게 할 수 있다.

연번|명칭|법정동|난방|관리사무실|팩스|준공일|방식|연차|동수|방광|요일|시간|금액|선입|문어|매수|1차|2차|비고
--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--
ㅇ|

## 피드백

비판과 재비판을 함 해보려고 한다.

**원하는 지역의 아파트 코드만 가져온다**

1. 필요한 건 오직 특정 시군구 정보 뿐이다.
 - 아니다. 특정 시군구 법정코드와 아파트 코드가 필요하다.
2. 특정 시군구 아파트 코드만 필요하다
 - 가능하려면 2.x 메가짜리 파일에서 특정 동을 검색해서 그부분만 잘라서 xml 로 만든 뒤에 매번 csv 임포트 해줘야 한다.

즉, sql 에 현재 가진 모든 법정코드와 주소를 테이블로 넣어두고 아파트 정보가 디테일하게 담긴 테이블에 정보를 넣어두는 것이 좋다.

**아파트 코드만 얻어오면 아파트 정보를 얻을 수 있다.**

- 그러나 읽어온 아파트 정보는 처리가 필요하다.
- 게다가 영문으로 된 정보가 한글로 무엇인지 번역될 필요가 있다.

```
http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255
```

In [2]:
  import ( "io/ioutil";"net/http";"strings";"fmt";"bytes" )

    res, err := http.Get("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255")
    defer res.Body.Close()
    body, err := ioutil.ReadAll(res.Body); buf := bytes.NewBuffer(body); json := buf.String(); fmt.Println(json[:300])
  	

{"resultMap_match":{"KAPT_CODE":"A10027255","TOWN_CODE":20306348},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"-","SUBWAY_LINE":"7호선, 인천선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"분무식","KAPT_PE5AREA":0,"KAP
301
<nil>


# 구현 

## 모든 행정동 코드 정보를 테이블에 넣기

### 파일을 읽어서 존재하는 정보만 배열로 준비하기

In [1]:
import ("encoding/csv"; "fmt"; "os"; "github.com/kniren/gota/dataframe";)

// 접속 생성 및  닫기 지연 걸기
 csvFile, err := os.Open("./법정동코드_전체자료.txt")
 defer csvFile.Close()

// 일단 리더생성 후 쉼표 옵션 기호를 탭으로 설정
 reader := csv.NewReader(csvFile)
 reader.Comma = '\t' // Use tab-delimited instead of comma <---- here!
 reader.FieldsPerRecord = -1

// 모두 읽어제껴버림
 arrayData, err := reader.ReadAll()

// 헤더를 제외한 데이터만 배열에 담기
var bjdArrData [][]string
ind := 0
for index, row := range arrayData{
  
    if (index == 0){
        // bjdArrData = append(filteredArrData, row[:2])
        // index가 0 이면 다시 for 룹 시작으로 가서 index 1 부터 처리
        // 즉 헤더 빼고 데이터처리만 하라는 말
        continue
    }else if(row[2]=="존재"){
        bjdArrData = append(bjdArrData, row[:2])
    }
}

fmt.Println(bjdArrData[:4])

[[1100000000 서울특별시] [1111000000 서울특별시 종로구] [1111010100 서울특별시 종로구 청운동] [1111010200 서울특별시 종로구 신교동]]
168
<nil>


### 테이블 생성 후 배열 자료를 추가하기 

생성

In [2]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Println(err)
}
defer db.Close()

createtable:= `CREATE TABLE 법정동 (
                법정동코드 bigint NOT NULL DEFAULT 0,
                법정동명 varchar(30)  NOT NULL DEFAULT ''
              )`

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err1 := db.Prepare(createtable)

defer stmt.Close()


_, err = stmt.Exec()
if err != nil {
    fmt.Println(err.Error())
}

pq: relation "법정동" already exists


벌크 임포트(대량 가져오기) https://godoc.org/github.com/lib/pq#hdr-Bulk_imports

prepared statement http://go-database-sql.org/prepared.html


In [3]:
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("법정동", "법정동코드", "법정동명"))
if err != nil {
	fmt.Println(err)
}

for _, bjd := range bjdArrData {
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
        _, err = stmt.Exec(bjd[0], bjd[1])
	if err != nil {
		fmt.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}

## 특정 키워드를 입력하면 해당하는 모든 법정동 코드 가져오는 함수 만들기

### 함수 설계 (mock)

In [33]:
func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    
    법정동코드결과 = append(법정동코드결과, 1001, 1002, 1003)
    
    return 법정동코드결과
}

fmt.Println(getBjd("지역키워드"))

[1001 1002 1003]
17
<nil>


### 쿼리 설계

In [36]:
// db 접속 포인터를 얻고
db, err := sql.Open("postgres", dbinfo)

if err != nil { log.Println(err)}
defer db.Close()

// Query the database.
rows, err := db.Query(`
    SELECT 법정동코드
    FROM 법정동
    WHERE 법정동명 LIKE $1
    `, "%서울%")

if err != nil { log.Println(err)}
defer rows.Close()
    
fmt.Println("질의 날리기")

법정동코드결과 := []string{}

for rows.Next() {
    
    코드하나 := ""

if err := rows.Scan(&코드하나); err != nil {
log.Println(err)}
 
   법정동코드결과 =  append(법정동코드결과,코드하나)
// fmt.Printf("%.2d, \n", 법정동코드)
}
    
fmt.Println("출력하기", 법정동코드결과[:4])
    

질의 날리기
출력하기 [1100000000 1111000000 1111010100 1111010200]
59
<nil>


### 지역 키워드에 따라 법정동 코드를 가져오는 함수 완성

In [4]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    db, err := sql.Open("postgres", dbinfo)
    if err != nil { log.Println(err)}
    defer db.Close()
    
    // Query the database.
    rows, err := db.Query(`SELECT 법정동코드 FROM 법정동 WHERE 법정동명 LIKE $1`,
                      "%"+ areaName +"%")
    for rows.Next() {    
        코드하나 := 0
        if err := rows.Scan(&코드하나); err != nil {log.Println(err)}
        법정동코드결과 =  append(법정동코드결과,코드하나)
    }
    
    return 법정동코드결과
}

fmt.Println(getBjd("구례")[:4])

[46730250 46730310 46730320 46730330]
38
<nil>


## 법정동을 주면 아파트 코드를 가져오는 함수 작성

### 함수 설계 - Mock

In [5]:
func getAptCode(bjdCode int) []string {
    
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    아파트코드결과 = append(아파트코드결과, "A1001", "A1002", "A1003")
    
    return 아파트코드결과
}

fmt.Println(getAptCode(263802006002))

[A1001 A1002 A1003]
20
<nil>


### get 요청과 처리 설계

#### 요청 주소 만들기

In [5]:
import (
    "fmt"
    "log"
    "net/http"
    "os"
 "net"
  "net/url"
    "strconv"
)

// 인증키 디코딩
s := "http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?loadCode=263802006002&ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D"
u, err := url.Parse(s)
m, _ := url.ParseQuery(u.RawQuery)
serviceKey := m["ServiceKey"][0]
fmt.Println(serviceKey)

// 인증키를 넘기면 쿼리 주소를 반환
func getAptlistUrl(targetUrl string, serviceKey string, bjdCode int) string{
    
    req, err := http.NewRequest("GET", targetUrl, nil)
    if err != nil {
        log.Print(err)
        //os.Exit(1)
    }

    bjdCodeStr := strconv.Itoa(bjdCode)
    q := req.URL.Query()
    q.Add("ServiceKey", serviceKey)    
    q.Add("loadCode",bjdCodeStr) 
    req.URL.RawQuery = q.Encode()
    url := req.URL.String()
    fmt.Println(req.URL.String())
    return  url   
}
bjdCode := 263802006002
aptListurl := "http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList"
queryUrl := getAptlistUrl(aptListurl, serviceKey, bjdCode)

9sfZOPVes/SD+woe/T+hLuNcrJUi0XE/LEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg==
http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D&loadCode=263802006002


#### 주소 요청 결과로 구조체 만들기

위 주소를 받아서 xml 로 언마셜한 다음에 json 으로 마셜한다.

In [6]:
import(
    "encoding/json"
    "encoding/xml"
    "io/ioutil"
    "bytes"
    "net/http"
    "reflect"
    "log"
    "fmt"
)

func getXmlResponse(queryUrl string) []uint8{

    response, err := http.Get(queryUrl)
    if err != nil {
        log.Fatal(err)
    }

    defer response.Body.Close()

    body, err := ioutil.ReadAll(response.Body)
    if err != nil {
        log.Fatal(err)
    }
    
    // fmt.Println(reflect.TypeOf(body)) 
    // []uint8
   
    strBody := bytes.NewBuffer(body)

    fmt.Println(strBody.String())
    
    return body
}

respBody := getXmlResponse(queryUrl)


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><kaptCode>A10027875</kaptCode><kaptName>괴정 경성스마트W아파트</kaptName></item><item><kaptCode>A60470801</kaptCode><kaptName>괴정엔스타</kaptName></item><item><kaptCode>A60473407</kaptCode><kaptName>신동양</kaptName></item><item><kaptCode>A60481303</kaptCode><kaptName>괴정일동지에닌아파트</kaptName></item><item><kaptCode>A60485510</kaptCode><kaptName>자유</kaptName></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>5</totalCount></body></response>


구조는 다음과 같다.

In [ ]:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
    <response>
        <header>
            <resultCode>00</resultCode>
            <resultMsg>NORMAL SERVICE.</resultMsg>
        </header>
        <body>
            <items>
                <item>
                    <kaptCode>A10027875</kaptCode>
                    <kaptName>괴정 경성스마트W아파트</kaptName>
                </item>
                <item><kaptCode>A60470801</kaptCode><kaptName>괴정엔스타</kaptName></item>
                <item><kaptCode>A60473407</kaptCode><kaptName>신동양</kaptName></item>
                <item><kaptCode>A60481303</kaptCode><kaptName>괴정일동지에닌아파트</kaptName></item>
                <item><kaptCode>A60485510</kaptCode><kaptName>자유</kaptName></item>
            </items>
            <numOfRows>10</numOfRows>
            <pageNo>1</pageNo>
            <totalCount>5</totalCount>
        </body>
    </response>
668
<nil>

구조체는 다음과 같음..  

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/

네스티드로 구조체 짤 수도 있다.
https://stackoverflow.com/questions/32125816/parsing-xml-in-golang-unmarshaling

In [7]:
import(
    "encoding/json"
    "encoding/xml"
    "bytes"
"reflect")


type AptCodeResponse struct {
    XMLName xml.Name `xml:"response" json:"-"`
    Header AptCodeHeader  `xml:"header" json:"header"`
    Body AptCodeBody `xml:"body" json:"body"`
}

type AptCodeHeader struct{
    XMLName xml.Name `xml:"header" json:"-"`
    ResultCode int  `xml:"resultCode" json:"resultCode"`
    ResultMsg string `xml:"resultMsg" json:"resultMsg"`
}

type AptCodeBody struct {
    Items AptCodeItems `xml:"items" json:"items"`
    XMLName xml.Name `xml:"body" json:"-"`
    NumOfRows int  `xml:"numOfRows" json:"numOfRows"`
    PageNo int `xml:"pageNo" json:"pageNo"`    
    TotalCount int `xml:"totalCount" json:"totalCount"`    
}
// 아래처럼 하위 구조를 직접 파도 되고
//     ItemList struct{
//         Item []struct{
//             KaptCode string  `xml:"kaptCode" json:"kaptCode"`
//             KaptName string  `xml:"kaptName" json:"kaptName"`
//         }`xml:"item" json:"item"`
//     }`xml:"items" json:"items"`
// }

// items 밑에 여러 item 을 만들어도 된다.
type AptCodeItems struct {
    XMLName xml.Name `xml:"items" json:"-"`
    ItemList []AptCodeItem `xml:"item" json:"item"`
}


type AptCodeItem struct {
    XMLName xml.Name `xml:"item" json:"-"`
    KaptCode string  `xml:"kaptCode" json:"kaptCode"`
    KaptName string  `xml:"kaptName" json:"kaptName"`
}


func unMarshalAptXml(responseBody []byte) AptCodeResponse {
    
    var data AptCodeResponse
    xml.Unmarshal([]byte(responseBody), &data)
    fmt.Println(data)
    // jsonData, _ := json.Marshal(data)
    // fmt.Println(string(jsonData))

 return data
}

xmlData := unMarshalAptXml(respBody)


{{ response} {{ header} 0 NORMAL SERVICE.} {{{ items} [{{ item} A10027875 괴정 경성스마트W아파트} {{ item} A60470801 괴정엔스타} {{ item} A60473407 신동양} {{ item} A60481303 괴정일동지에닌아파트} {{ item} A60485510 자유}]} { body} 10 1 5}}


#### XML 로부터 아파트 코드 추출하기

In [8]:
import(
    "encoding/json"
    "encoding/xml"
    "bytes"
"reflect"
"fmt")

serviceKey := "9sfZOPVes/SD+woe/T+hLuNcrJUi0XE/LEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg=="

//     queryUrl := getQueryUrl(serviceKey)
//     respBody := getXmlResponse(queryUrl)
//     xmlData := unMarshalXml(respBody)
// fmt.Println(xmlData.Body.Items)

func getAptCode(aptXmlData AptCodeResponse) []string {
        
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    items := xmlData.Body.Items.ItemList
    
    for _,v := range items{
        // fmt.Println(k,v)
        아파트코드결과 = append(아파트코드결과, v.KaptCode )
    }
    
    
    
    return 아파트코드결과
}
aptCodes := getAptCode(xmlData)
fmt.Println(aptCodes)

[A10027875 A60470801 A60473407 A60481303 A60485510]
52
<nil>


## 아파트 기본 정보 담을 테이블 생성하기

### 아파트 기본 정보 xml 정보 확인 + 쿼리 작성 + 테이블 생성


response body 만 xml 로 바꿔서 xml 파싱한다.
https://tutorialedge.net/golang/parsing-xml-with-golang/

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/


받게되는 정보에 관한 설명
https://www.data.go.kr/subMain.jsp#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTIkQF5wdWJsaWNEYXRhRGV0YWlsUGs9dWRkaTphMTJjNmU3Ni04OTgyLTRlM2UtYWZlMi05NDQxMmM3ZDBhNzkkQF5tYWluRmxhZz10cnVl

In [9]:
import(
    "encoding/json"
    "encoding/xml"
    "io/ioutil"
    "bytes"
    "net/http"
    "reflect"
    "log"
    "fmt"
)

serviceKey := "9sfZOPVes/SD+woe/T+hLuNcrJUi0XE/LEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg=="

func getAptDetailtUrl(targetUrl string, serviceKey string, kaptCode string) string{
    
    req, err := http.NewRequest("GET", targetUrl, nil)
    if err != nil {
        log.Print(err)
        //os.Exit(1)
    }

    q := req.URL.Query()
    q.Add("ServiceKey", serviceKey)    
    q.Add("kaptCode",  kaptCode)
    req.URL.RawQuery = q.Encode()
    url := req.URL.String()
    // fmt.Println(req.URL.String())
    return  url   
}

kaptCode := "A10027875"
aptListurl := "http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo"
queryUrl := getAptDetailtUrl(aptListurl, serviceKey, kaptCode)

In [10]:
respBody := getXmlResponse(queryUrl)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><item><bjdCode>2638010100</bjdCode><codeAptNm>아파트</codeAptNm><codeHallNm>혼합식</codeHallNm><codeHeatNm>개별난방</codeHeatNm><codeMgrNm>위탁관리</codeMgrNm><codeSaleNm>분양</codeSaleNm><hoCnt>182</hoCnt><kaptAcompany>(주)경성리츠</kaptAcompany><kaptAddr>부산광역시 사하구 괴정동 258 괴정 경성스마트W아파트</kaptAddr><kaptBcompany>(주)경성리츠</kaptBcompany><kaptCode>A10027875</kaptCode><kaptDongCnt>3</kaptDongCnt><kaptFax>051-294-9364</kaptFax><kaptMarea>15040.1634</kaptMarea><kaptMparea_135>0</kaptMparea_135><kaptMparea_136>0</kaptMparea_136><kaptMparea_60>182</kaptMparea_60><kaptMparea_85>0</kaptMparea_85><kaptName>괴정 경성스마트W아파트</kaptName><kaptTarea>19324.6751</kaptTarea><kaptTel>051-294-9363</kaptTel><kaptUrl> </kaptUrl><kaptUsedate>20150806</kaptUsedate><kaptdaCnt>182</kaptdaCnt><privArea>9014.0338</privArea></item></body></response>


In [11]:
type AptDetailResponse struct {
    XMLName xml.Name `xml:"response" json:"-"`
    Header AptDetailHeader  `xml:"header" json:"header"`
    Body AptDetailBody `xml:"body" json:"body"`
}

type AptDetailHeader struct{
    XMLName xml.Name `xml:"header" json:"-"`
    ResultCode int  `xml:"resultCode" json:"resultCode"`
    ResultMsg string `xml:"resultMsg" json:"resultMsg"`
}

type AptDetailBody struct {
    XMLName xml.Name `xml:"body" json:"-"`
    Item AptDetailItem `xml:"item" json:"item"`
}

type AptDetailItem struct {
    XMLName xml.Name `xml:"item" json:"-"`
    법정동코드 int  `xml:"bjdCode" json:"bjdCode"`
    아파트코드	string `xml:"kaptCode" json:"kaptCode"`
    아파트명	string `xml:"kaptName" json:"kaptName"`
    법정동주소	string `xml:"kaptAddr" json:"kaptAddr"`
    분양형태	string `xml:"codeSaleNm" json:"codeSaleNm"`
    난방방식	string `xml:"codeHeatNm" json:"codeHeatNm"`
    건축물대장상연면적	float64 `xml:"kaptTarea" json:"kaptTarea"`
    동수	int `xml:"kaptDongCnt" json:"kaptDongCnt"`
    세대수	int `xml:"kaptdaCnt" json:"kaptdaCnt"`
    시공사	string `xml:"kaptBcompany" json:"kaptBcompany"`
    시행사	string `xml:"kaptAcompany" json:"kaptAcompany"`
    관리사무소연락처	string `xml:"kaptTel" json:"kaptTel"`
    관리사무소팩스	string `xml:"kaptFax" json:"kaptFax"`
    홈페이지주소	string `xml:"kaptUrl" json:"kaptUrl"`
    단지분류	string `xml:"codeAptNm" json:"codeAptNm"`
    도로명주소	string `xml:"doroJuso" json:"doroJuso"`
    호수	string `xml:"hoCnt" json:"hoCnt"`
    관리방식	string `xml:"codeMgrNm" json:"codeMgrNm"`
    복도유형	string `xml:"codeHallNm" json:"codeHallNm"`
    사용승인일	int `xml:"kaptUsedate" json:"kaptUsedate"`
    관리비부과면적	float64 `xml:"kaptMarea" json:"kaptMarea"`
    전용면적별세대현황60	int `xml:"kaptMparea_60" json:"kaptMparea_60"`
    전용면적별세대현황85	int `xml:"kaptMparea_85" json:"kaptMparea_85"`
    전용면적별세대현황135	int `xml:"kaptMparea_135" json:"kaptMparea_135"`
    전용면적별세대현황136	int `xml:"kaptMparea_136" json:"kaptMparea_136"`
    단지전용면적합 float64 `xml:"privArea" json:"privArea"`
 }

func unMarshalBjdXml(responseBody []byte) AptDetailResponse {
    
    var data AptDetailResponse
    xml.Unmarshal([]byte(responseBody), &data)
    // fmt.Println(data)
    // jsonData, _ := json.Marshal(data)
    // fmt.Println(string(jsonData))

 return data
}

xmlData := unMarshalBjdXml(respBody)

## 특정 주소 키워드를 주면 지역 내 아파트정보를 반환하는 함수 만들기

In [ ]:
import "strconv"

// 지역 키워드를 주면, 아파트 정보들을 반환한다.
func getAptBaseInfo(areaKeyword string) []AptDetailItem {
    
    // prepare const
    serviceKey := "9sfZOPVes/SD+woe/T+hLuNcrJUi0XE/LEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg=="
    aptListurl := "http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList"
    //aptDetailurl := "http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo"
    
    // bucket
    var aptInformations []AptDetailItem
    
    
    fmt.Println("법정동 목록 시작","-----------")
    
    // 법정동 주소의 목록을 반환
    법정동목록 := getBjd(areaKeyword)
    
    fmt.Println(법정동목록)
    
    fmt.Println("법정동 목록 완료","-----------")
    
    // 법정동 하나씩 마다
    for _, bjdCode := range 법정동목록{
        
        
        fmt.Println(bjdCode,"아파트 목록 조회 시작","-----------")
        
        queryUrl := getAptlistUrl(aptListurl, serviceKey, bjdCode)
        respBody := getXmlResponse(queryUrl)
        _ = respBody
        //xmlData := unMarshalAptXml(respBody)
        // _ = xmlData
        
//         fmt.Println("아파트 목록 조회 완료","-----------")
        
//         fmt.Println("아파트 목록 마샬링 시작","-----------")
        
//         // 여러 아파트 코드를 알아내서
//         aptCodes := getAptCode(xmlData)

        
//         fmt.Println("아파트 목록 마샬링 완료","-----------")
        
//         // 아파트 코드 하나마다
        
//         fmt.Println("아파트 정보 조회 시작","-----------")
//         for _, kaptCode := range aptCodes{
//             queryUrl := getAptDetailtUrl(aptDetailurl, serviceKey, kaptCode)
//             respBody := getXmlResponse(queryUrl)
//             xmlData := unMarshalBjdXml(respBody)
            
//             // 정보를 알아낸 다음 차곡차곡 끼워 넣는다.
//             aptInformations = append(aptInformations, xmlData.Body.Item)
//             fmt.Println(xmlData.Body.Item.아파트명, "처리완료")
//         }
        
        fmt.Println(len(aptInformations),"처리 완료")
    }

    return aptInformations
}

x := getAptBaseInfo("영등포구")
fmt.Println(x)

법정동 목록 시작 -----------
[1156000000 1156010100 1156010200 1156010300 1156010400 1156010500 1156010600 1156010700 1156010800 1156010900 1156011000 1156011100 1156011200 1156011300 1156011400 1156011500 1156011600 1156011700 1156011800 1156011900 1156012000 1156012100 1156012200 1156012300 1156012400 1156012500 1156012600 1156012700 1156012800 1156012900 1156013000 1156013100 1156013200 1156013300 1156013400 1156000000 1156010100 1156010200 1156010300 1156010400 1156010500 1156010600 1156010700 1156010800 1156010900 1156011000 1156011100 1156011200 1156011300 1156011400 1156011500 1156011600 1156011700 1156011800 1156011900 1156012000 1156012100 1156012200 1156012300 1156012400 1156012500 1156012600 1156012700 1156012800 1156012900 1156013000 1156013100 1156013200 1156013300 1156013400 1156000000 1156010100 1156010200 1156010300 1156010400 1156010500 1156010600 1156010700 1156010800 1156010900 1156011000 1156011100 1156011200 1156011300 1156011400 1156011500 1156011600 1156011700 115601180

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>
0 처리 완료
1156011500 아파트 목록 조회 시작 -----------
http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D&loadCode=1156011500
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>
0 처리 완료
1156011600 아파트 목록 조회 시작 -----------
http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D&loadCode=1156011600
<?xml version="1

In [12]:
// 법정동 주소의 목록을 반환
법정동목록 := getBjd("전라남도 구례군 구례읍 봉동리")

In [13]:
// 서비스키를 가지고 URL 생성
serviceKey := "9sfZOPVes/SD+woe/T+hLuNcrJUi0XE/LEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg=="
aptListurl := "http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList"
bjdCode := 4673025021
queryUrl := getAptlistUrl(aptListurl, serviceKey, bjdCode)

http://apis.data.go.kr/1611000/AptListService/getRoadnameAptList?ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D&loadCode=4673025021


In [ ]:
// url 에서 응답 반환
respBody := getXmlResponse(queryUrl)
xmlData := unMarshalAptXml(respBody)
// aptCodes := getAptCode(xmlData)

fmt.Println(aptCodes)

In [22]:
kaptCode := aptCodes[0]
aptDetailurl := "http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo"
queryUrl := getAptDetailtUrl(aptListurl, serviceKey, kaptCode)

xmlData := unMarshalBjdXml(respBody)

import "reflect"
reflect.TypeOf(xmlData.Body.Item)

http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo?ServiceKey=9sfZOPVes%2FSD%2Bwoe%2FT%2BhLuNcrJUi0XE%2FLEf76rL2wuipBUT0GLQD4kEs4DISzyoOTkaRn4Ap52qnkpCUwElILg%3D%3D&kaptCode=A10027875
{{ response} {{ header} 0 NORMAL SERVICE.} {{ body} {{ item} 2638010100 A10027875 괴정 경성스마트W아파트 부산광역시 사하구 괴정동 258 괴정 경성스마트W아파트 분양 지역난방 19324.6751 3 182 (주)경성리츠 (주)경성리츠 051-294-9363 051-294-9364   아파트  182 위탁관리 혼합식 20150806 15040.1634 182 0 0 0 9014.0338}}}
lgo_exec.AptDetailItem
